In [1]:
from proj1_helpers import *
from implementations import *
from auxiliary import *
import pickle
import matplotlib.pyplot as plt
import datetime
import random
import copy

%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Data loading


### Training sets

In [2]:
easy_data_set = False # changer pour sélectionner un set de dim (1000,2) ou le vrai set
data_subset = True # the data is divided in 8 subsets

if data_subset:
    r=8
else:
    r=1

# declare all variables as lists
ids=[0]*r
y=[0]*r
X=[0]*r
x=[0]*r
tx=[0]*r
mean_x=[0]*r
std_x=[0]*r
N=[0]*r
D=[0]*r
prop=[0]*r

if easy_data_set:
    ids[0], y[0], X[0] = load_easy_data(sub_sample=False)
else:
    ids[0], y[0], X[0] = load_csv_data('../data/train.csv')
    len_tot = X[0].shape[0]
    if data_subset:
        ids, y, X = split_data_boson(ids[0], X[0], y[0])

    else:
        # Only 100% clean data
        ranges = [(1,4), (7,12), (13,23), (29,30)]
        keep_idx = build_idx(ranges)
        X[0] = X[0][:,keep_idx]


for r in range(len(X)):
    x[r], mean_x[r], std_x[r] = standardize(X[r])    # standardize
    clean_data = [x for x in x[r] if not -999. in x] # check tidyness of data
    print(len(clean_data)/len(x[r])*100) 
    tx[r] = build_poly(x[r], 1)                      # add column of 1's
    N[r] = x[r].shape[0]
    D[r] = x[r].shape[1]
    prop[r] = len(x[r])/len_tot
    print(prop[r])


100.0
0.104492
100.0
0.29516
100.0
0.030248
100.0
0.279928
100.0
0.011808
100.0
0.189708
100.0
0.005908
100.0
0.082748


### Testing sets

In [3]:
if data_subset:
    r=8
else:
    r=1

# declare variables
ids_ukn=[0]*r
y_ukn=[0]*r
X_ukn=[0]*r
x_ukn=[0]*r
tx_ukn=[0]*r
mean_x_ukn=[0]*r
std_x_ukn=[0]*r


# takes time, run only when needed
ids_ukn[0], y_ukn[0], X_ukn[0] = load_csv_data('../data/test.csv')

if data_subset:
    ids_ukn, y_ukn, X_ukn = split_data_boson(ids_ukn[0], X_ukn[0], y_ukn[0])
else:
    ranges = [(1,4), (7,12), (13,23), (29,30)]
    keep_idx = build_idx(ranges)
    X_ukn[0] = X_ukn[0][:,keep_idx]


for r in range(len(X_ukn)):
    x_ukn[r], mean_x_ukn[r], std_x_ukn[r] = standardize(X_ukn[r])    # standardize
    clean_data = [x for x in x_ukn[r] if not -999. in x]     # check tidyness of data
    print(len(clean_data)/len(x_ukn[r])*100) 
    tx_ukn[r] = build_poly(x_ukn[r], 1)                      # add column of 1's

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0


Dividing data in 8 subsets

# 100% Training
On s'entraine sur 100% des datas

In [4]:
if data_subset:
    r=8
else:
    r=1

loss=[0]*r
w=[0]*r
w_LS=[0]*r
w_LS_GD=[0]*r
w_LS_SGD=[0]*r
w_LS_degree=[0]*r
w_ridge_reg=[0]*r
w_LR=[0]*r

ratio_error_train=[0]*r
ratio_error_test=[0]*r


## Least squares

In [71]:
for r in range(len(x)):
    tx[r] = build_poly(x[r], 1)
    w_LS[r], loss[r] = least_squares(y[r],tx[r])
print(loss)

[0.022707652148050421, 0.078046547757038051, 0.035009560260221242, 0.098022285159922506, 0.041224422936763094, 0.089890162893709352, 0.028470716296341923, 0.092875527585607548]


## Least squares (GD)

In [24]:
# w_initial = w_LS

max_iters = 100
gammas = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.5]
losses = np.zeros([len(gammas)])
ws = []
degree=1
exp=0

for r in range(len(x)):
    tx[r] = build_poly(x[r], degree)

for gamma in gammas:
    for r in range(len(x)):
        w_initial = np.array([0]*tx[r].shape[1])
        w[r], loss[r] = least_squares_GD(y[r], tx[r], w_initial, max_iters, gamma)
        losses[exp] += loss[r]*prop[r]
    ws.append(copy.copy(w))
    exp+=1

In [25]:
idx = np.argmin(losses) # get best experiment
w_LS_GD = ws[idx]

print("Take experiment {i} out of {tot} experiments, loss={l}, degree={deg}, gamma={ga}, max_iter={mi}".format(
           i=idx+1, tot=len(ws), l=losses[idx], deg=degree, ga=gammas[idx],mi=max_iters))


Take experiment 10 out of 11 experiments, loss=0.07942102497394102, degree=1, gamma=0.2, max_iter=100


## Least squares (SGD)

In [34]:
max_iters = 1000
gammas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.4, 0.5]
batch_size = 1
losses = np.zeros([len(gammas)])
ws = []
exp=0
degree=1

for r in range(len(x)):
    tx[r] = build_poly(x[r], degree)

for gamma in gammas:
    for r in range(len(x)):
        w_initial = np.array([0]*(tx[r].shape[1]))
        w[r], loss[r] = least_squares_SGD(y[r], tx[r], w_initial, batch_size, max_iters, gamma)
        losses[exp] += loss[r]*prop[r]
    ws.append(w)

/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:7: RuntimeWarning: overflow encountered in square
  return 1/2 * np.mean(e**2)
/Users/philippe/Desktop/ML_Projects/project1/phil/implementations.py:44: RuntimeWarning: invalid value encountered in subtract
  w = w - gamma * gradients


In [35]:
idx = np.argmin(losses) # get best experiment
w_LS_SGD = ws[idx]

print("Take experiment {i} out of {tot} experiments, loss={l}, gamma={g}".format(
           i=idx+1, tot=len(ws), l=losses[idx], g=gammas[idx]))



[ nan   0.   0.   0.   0.   0.   0.   0.]
Take experiment 1 out of 8 experiments, loss=nan, gamma=1e-05


## Polynomial regression

In [36]:
"""Constructing the polynomial basis function expansion of the data,
   and then running least squares regression."""
# define parameters
degrees = [1, 3, 7, 10, 11, 12, 13, 14, 15]
losses = []
ws = []
ratio_errs = np.zeros([len(degrees)])
deg = []
exp=0
        
for ind, degree in enumerate(degrees):
    for r in range(len(x)):
        tx = build_poly(x[r], degree)  # form the data to do polynomial regression.
        # least square and calculate RMSE
        
        w[r], loss_degree_LS = least_squares(y[r], tx)
        error, ratio_error = compute_classification_error(y[r], tx, w[r], logistic_reg=False)
        ratio_errs[exp] += ratio_error*prop[r]
    
    
    losses.append(loss_degree_LS)
    ws.append(copy.copy(w))
    deg.append(degree)
    print("{exp}: Processing {i}th experiment, degree={d}, loss={loss:.3f}, err_ratio={err:.3f}".format(
           exp=exp, i=ind, d=degree, loss=loss_degree_LS, err=ratio_errs[exp]))
    exp+=1


0: Processing 0th experiment, degree=1, loss=0.093, err_ratio=0.235
1: Processing 1th experiment, degree=3, loss=0.080, err_ratio=0.195
2: Processing 2th experiment, degree=7, loss=0.073, err_ratio=0.177
3: Processing 3th experiment, degree=10, loss=0.067, err_ratio=0.171
4: Processing 4th experiment, degree=11, loss=0.067, err_ratio=0.169
5: Processing 5th experiment, degree=12, loss=0.065, err_ratio=0.168
6: Processing 6th experiment, degree=13, loss=0.067, err_ratio=0.188
7: Processing 7th experiment, degree=14, loss=0.095, err_ratio=0.279
8: Processing 8th experiment, degree=15, loss=20.948, err_ratio=0.236


In [44]:
idx = np.argmin(ratio_errs) # get best experiment
w_degree_LS = ws[idx]


print("Take experiment {i} out of {tot} experiments, degree={d}, err_ratio={err}".format(
           i=idx, tot=len(ws), d=deg[idx], err=ratio_errs[idx]))


Take experiment 5 out of 9 experiments, degree=12, err_ratio=0.16785200000000003


## Ridge Regression
To do with subsets

In [53]:
degrees = [1, 3, 7, 12]
lambdas = np.logspace(-5, 2, 15)
    
ratio_errs = np.zeros([len(degrees)*len(lambdas)])
ws = []
deg=[]
exp=0

for degree in degrees:
    for r in range(len(x)):
        tx[r] = build_poly(x[r], degree)

    for ind, lambda_ in enumerate(lambdas):
        for r in range(len(x)):
            # form train and test data with polynomial basis function
            # ridge regression with a given lambda
            w[r], loss[r] = ridge_regression(y[r], tx[r], lambda_)
            error, ratio_error = compute_classification_error(y[r], tx[r], w[r], logistic_reg=False)
            ratio_errs[exp] += ratio_error*prop[r]

        ws.append(copy.copy(w))
    
        deg.append(degree)

        print("{exp}: degree={d}, lambda={l:.3f}, err_ratio={err_ratio:.3f}".format(
            exp=exp, d=degree, l=lambda_ , err_ratio=ratio_errs[exp]))
        
        exp += 1

0: degree=1, lambda=0.000, err_ratio=0.235
1: degree=1, lambda=0.000, err_ratio=0.235
2: degree=1, lambda=0.000, err_ratio=0.235
3: degree=1, lambda=0.000, err_ratio=0.235
4: degree=1, lambda=0.001, err_ratio=0.235
5: degree=1, lambda=0.003, err_ratio=0.235
6: degree=1, lambda=0.010, err_ratio=0.236
7: degree=1, lambda=0.032, err_ratio=0.240
8: degree=1, lambda=0.100, err_ratio=0.256
9: degree=1, lambda=0.316, err_ratio=0.305
10: degree=1, lambda=1.000, err_ratio=0.339
11: degree=1, lambda=3.162, err_ratio=0.343
12: degree=1, lambda=10.000, err_ratio=0.343
13: degree=1, lambda=31.623, err_ratio=0.343
14: degree=1, lambda=100.000, err_ratio=0.343
15: degree=3, lambda=0.000, err_ratio=0.195
16: degree=3, lambda=0.000, err_ratio=0.195
17: degree=3, lambda=0.000, err_ratio=0.195
18: degree=3, lambda=0.000, err_ratio=0.195
19: degree=3, lambda=0.001, err_ratio=0.195
20: degree=3, lambda=0.003, err_ratio=0.196
21: degree=3, lambda=0.010, err_ratio=0.199
22: degree=3, lambda=0.032, err_ratio=

KeyboardInterrupt: 

In [49]:
idx = np.argmin(ratio_errs) # get best experiment
w_ridge_reg = ws[idx]


print("Take experiment {i} out of {tot} experiments, degree={d}, err_ratio={err:.3f}".format(
           i=idx, tot=len(ws), d=deg[idx], err=ratio_errs[idx]))

Take experiment 48 out of 60 experiments, degree=12, err_ratio=0.167


## Logistic regression

In [54]:
max_iters = 500
gammas = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
#gammas = [1e-5, 1e-4]
losses = np.zeros([len(gammas), len(x)])
errors = np.zeros([len(gammas)])
ws = []
degree = 3

for r in range(len(x)):
    tx[r] = build_poly(x[r], degree)

for idx_g, gamma in enumerate(gammas):
    print("Run experiment {i}/{tot}".format(i=idx_g+1, tot=len(gammas)))
    for r in range(len(x)):
        w_initial = np.array([0]*(tx[r].shape[1]))
        w[r], loss[r] = logistic_regression(y[r], tx[r], w_initial, max_iters, gamma)
        nb_errors, error_mean = compute_classification_error(y[r], tx[r], w[r], logistic_reg=True)
        errors[idx_g] += error_mean*prop[r]
    losses[idx_g, :] = loss[:]  
    ws.append(copy.copy(w))
print("done")

Run experiment 1/8
Run experiment 2/8
Run experiment 3/8
Run experiment 4/8
Run experiment 5/8
Run experiment 6/8
Run experiment 7/8
Run experiment 8/8
done


In [56]:
idx = np.argmin(errors) # get best experiment

w_LR = ws[idx]

print("Take experiments {i} out of {tot} experiments, error={er:.3f}, gamma={g}".format(
           i=idx+1, tot=len(ws),  er=errors[idx], g=gammas[idx]))


Take experiments 3 out of 8 experiments, error=0.188, gamma=1e-06


# Evaluate for test set
Sert à afficher le nombre d'erreurs

In [8]:
final_error_mean = 0.0

tx=[0]*8

for r in range(len(x)):
    tx[r] = build_poly(x[r], 3)
    w_selected = w_LR_CV[r]
    nb_errors, error_mean = compute_classification_error(y[r], tx[r], w_selected, logistic_reg=True)
    #print(nb_errors)
    final_error_mean += error_mean*prop[r]
print(final_error_mean)

0.186212


In [24]:
print(w_LS)
print(w_LS_GD)
print(w_LS_SGD)
print(w_degree_LS)
print(w_ridge_reg)
print(w_LR)
print(w_LR_CV)
print(w_reg_log_reg)

NameError: name 'w_LS' is not defined

# Cross validation
Without k-fold

## Cross validation with least squares
To do with subsets

In [43]:
seed = 6
degrees = [1, 3, 7, 11, 12, 13, 14]
exp = 0

loss_tr = []
loss_te = []
ratio_err_trains = []
ratio_err_tests = []
ws = []
deg=[]

k_fold = 4
k_indices = build_k_indices(y, k_fold, seed)

print(k_indices.shape)

for degree in degrees:
    phi = build_poly(x, degree)
    loss_tr_k_fold = []
    loss_te_k_fold = []
    ratio_err_trains_k_fold = []
    ratio_err_tests_k_fold = []
    ws_k_fold = []
    for k in range(k_fold):
        # split the data, and return train and test data: TODO
        phi_train, phi_test, y_train, y_test = split_data_k_fold(phi, y, k_indices, k)
            
        # calcualte weight through least square.: TODO
        w,loss = least_squares(y_train, phi_train)
        ws_k_fold.append(w)

        # calculate RMSE for train and test data,
        # and store them in rmse_tr and rmse_te respectively: TODO
        loss_tr_k_fold.append(compute_loss(y_train,phi_train,w), logistic_reg=False)
        loss_te_k_fold.append(compute_loss(y_test,phi_test,w), logistic_reg=False)
    
        error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
        error, ratio_error_test = compute_classification_error(y_test, phi_test, w)
        
        ratio_err_trains_k_fold.append(ratio_error_train)
        ratio_err_tests_k_fold.append(ratio_error_test)
        
    get_best = False # get best or get mean
        
    if get_best:
        idx = np.argmin(ratio_err_tests_k_fold) # get best experiment
        ws.append(ws_k_fold[idx])
        loss_tr.append(loss_tr_k_fold[idx])
        loss_te.append(loss_te_k_fold[idx])
        ratio_err_trains.append(ratio_err_trains_k_fold[idx])
        ratio_err_tests.append(ratio_err_tests_k_fold[idx])
    else:
        ws.append(sum(ws_k_fold)/len(ws_k_fold))
        loss_tr.append(sum(loss_tr_k_fold)/len(loss_tr_k_fold))
        loss_te.append(sum(loss_te_k_fold)/len(loss_te_k_fold))
        ratio_err_trains.append(sum(ratio_err_trains_k_fold)/len(ratio_err_trains_k_fold))
        ratio_err_tests.append(sum(ratio_err_tests_k_fold)/len(ratio_err_tests_k_fold))
        
    deg.append(degree)
        
    print("exp={e}, degree={d}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
        e=exp, d=degree, tr=loss_tr[exp], te=loss_te[exp], er_tr=ratio_err_trains[exp], er_te=ratio_err_tests[exp]))
    exp += 1


(4, 62500)
exp=0, degree=1, Train loss, err_ratio=(0.088,0.266), Test loss, err_ratio=(0.088,0.266)
exp=1, degree=3, Train loss, err_ratio=(0.081,0.239), Test loss, err_ratio=(1.627,0.240)
exp=2, degree=7, Train loss, err_ratio=(0.079,0.225), Test loss, err_ratio=(82447866.810,0.225)
exp=3, degree=11, Train loss, err_ratio=(0.075,0.206), Test loss, err_ratio=(355213321008866394112.000,0.207)
exp=4, degree=12, Train loss, err_ratio=(0.075,0.205), Test loss, err_ratio=(4204608108250730594304.000,0.206)
exp=5, degree=13, Train loss, err_ratio=(0.077,0.210), Test loss, err_ratio=(24543286522935610108280832.000,0.211)
exp=6, degree=14, Train loss, err_ratio=(0.093,0.232), Test loss, err_ratio=(82609134220373679177004482560.000,0.232)


## Cross validation with ridge regression

In [ ]:
lambdas = np.logspace(-5, 2, 15)

ratio = 0.9
degrees = [1, 3, 7, 12]

# split the data, and return train and test data
    
    
loss_tr = []
loss_te = []
ratio_err_trains = []
ratio_err_tests = []
ws = []
deg=[]

for degree in degrees:
    for ind, lambda_ in enumerate(lambdas):
        for r in range(len(x)):
            # form train and test data with polynomial basis function
            x_train, x_test, y_train, y_test = split_data(x[r], y[r], ratio)
            phi_train = build_poly(x_train, degree)
            phi_test = build_poly(x_test, degree)
            # ridge regression with a given lambda
            w[r], loss = ridge_regression(y_train, phi_train, lambda_)
        ws.append(w)
        loss_tr.append(compute_loss(y_train,phi_train,w))
        loss_te.append(compute_loss(y_test,phi_test,w))
    
        error, ratio_error_train = compute_classification_error(y_train, phi_train, w, logistic_reg=False)
        error, ratio_error_test = compute_classification_error(y_test, phi_test, w, logistic_reg=False)
    
        ratio_err_trains.append(ratio_error_train)
        ratio_err_tests.append(ratio_error_test)
        deg.append(degree)

        print("proportion={p}, degree={d}, lambda={l:.3f}, Train loss,err_ratio=({tr:.3f},{err_ratio_tr:.3f}), Test loss,err_ratio=({te:.3f},{err_ratio_te:.3f})".format(
            p=ratio, d=degree, l=lambda_ ,tr=loss_tr[ind], te=loss_te[ind], err_ratio_tr=ratio_error_train, err_ratio_te=ratio_error_test))

## Cross validation with logistic regression

In [23]:
def logistic_regression_split(x, y, degree, ratio, gamma, seed):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data(x, y, ratio, seed)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    
    w_initial = np.array([0]*(phi_train.shape[1]))
    max_iters = 1500

    
    # calcualte weight through least square.: TODO
    w,loss_tr = logistic_regression(y_train, phi_train, w_initial, max_iters, gamma)
    
    loss_te = compute_logistic_loss(y_test, phi_test, w)
    # compute error and loss for train and test data
    error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
    error, ratio_error_test = compute_classification_error(y_test, phi_test, w)

    
    #print("proportion={p}, degree={d}, gamma={g}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
    #      p=ratio, d=degree, g=gamma, tr=loss_tr, te=loss_te, er_tr=ratio_error_train, er_te=ratio_error_test))
    return w, ratio_error_train, ratio_error_test


In [24]:
seed = 6
degrees = [2,3,4,5,6]
split_ratios = [0.2]
gammas = [1e-6, 1e-5, 1e-4, 1e-3]

ws = []
ratio_err_trains = []
ratio_err_tests = []

print("There will be {exp} experiments".format(exp=len(degrees)*len(split_ratios)*len(gammas)))

for idx0, split_ratio in enumerate(split_ratios):
    for idx1, degree in enumerate(degrees):
        for idx2, gamma in enumerate(gammas):
            for r in range(len(x)): # compute the weight for each subset
                w[r], ratio_error_train[r], ratio_error_test[r] = logistic_regression_split(x[r], y[r], degree, split_ratio, gamma, seed)
            ws.append(copy.copy(w))
            ratio_err_trains.append(0)
            ratio_err_tests.append(0)
            for r in range(len(ratio_error_train)):
                ratio_err_trains[-1]+= ratio_error_train[r]*prop[r]
                ratio_err_tests[-1] += ratio_error_test[r]*prop[r]
            print("{exp}: proportion={p}, degree={d}, gamma={g}, Train err_ratio={er_tr:.3f}, Test err_ratio={er_te:.3f}".format(
                   exp=idx0*len(split_ratios)*len(degrees)*len(gammas)+idx1*len(degrees)*len(gammas)+idx2, p=split_ratio, d=degree, g=gamma, 
                   er_tr=ratio_err_trains[-1], er_te=ratio_err_tests[-1]))
print("done")

There will be 20 experiments
0: proportion=0.2, degree=2, gamma=1e-06, Train err_ratio=0.188, Test err_ratio=0.193
1: proportion=0.2, degree=2, gamma=1e-05, Train err_ratio=0.180, Test err_ratio=0.186
2: proportion=0.2, degree=2, gamma=0.0001, Train err_ratio=0.231, Test err_ratio=0.236
3: proportion=0.2, degree=2, gamma=0.001, Train err_ratio=0.231, Test err_ratio=0.236
20: proportion=0.2, degree=3, gamma=1e-06, Train err_ratio=0.185, Test err_ratio=0.186
21: proportion=0.2, degree=3, gamma=1e-05, Train err_ratio=0.179, Test err_ratio=0.182
22: proportion=0.2, degree=3, gamma=0.0001, Train err_ratio=0.229, Test err_ratio=0.232
23: proportion=0.2, degree=3, gamma=0.001, Train err_ratio=0.239, Test err_ratio=0.242
40: proportion=0.2, degree=4, gamma=1e-06, Train err_ratio=0.203, Test err_ratio=0.205
41: proportion=0.2, degree=4, gamma=1e-05, Train err_ratio=0.219, Test err_ratio=0.223
42: proportion=0.2, degree=4, gamma=0.0001, Train err_ratio=0.234, Test err_ratio=0.236
43: proportion=

In [25]:
idx = np.argmin(ratio_err_tests) # get best experiment
w_LR_CV = ws[idx]


idx_split = int(idx/(len(degrees)*len(gammas)))
idx_degree_gamma = idx%(len(degrees)*len(gammas))
idx_degree = int(idx_degree_gamma/len(gammas))
idx_gamma = (idx_degree_gamma%len(gammas))


print("Take experiments {i} out of {tot} experiments, error tr={er_tr:.3f}, error test={er_te:.3f}, split_ratio={spl}, degree={deg}, gamma={g}".format(
           i=idx, tot=len(ws), er_tr=ratio_err_trains[idx], er_te=ratio_err_tests[idx], 
           spl=split_ratios[idx_split],  deg=degrees[idx_degree], g=gammas[idx_gamma]))


Take experiments 5 out of 20 experiments, error tr=0.179, error test=0.182, split_ratio=0.2, degree=3, gamma=1e-05


## Cross validation with regularized logistic regression
To do with subsets

In [114]:
def reg_logistic_regression_split(x, y, degree, ratio, gamma, lambda_, seed):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data(x, y, ratio, seed)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    
    w_initial = np.array([0]*(phi_train.shape[1]))
    max_iters = 1000

    
    # calcualte weight through logistic regression
    w, loss_tr = reg_logistic_regression(y_train, phi_train, lambda_, w_initial, max_iters, gamma, SGD=False)
    
    loss_te = compute_logistic_loss(y_test, phi_test, w)
    # compute error and loss for train and test data
    error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
    error, ratio_error_test = compute_classification_error(y_test, phi_test, w)

    
    #print("prop={p}, deg={d}, g={g:.3f}, l={l:.3f}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
    #      p=ratio, d=degree, g=gamma, l=lambda_, tr=loss_tr, te=loss_te, er_tr=ratio_error_train, er_te=ratio_error_test))
    return w, ratio_error_train, ratio_error_test


In [125]:
seed = 6
degrees = [2, 3, 4, 5]
split_ratios = [0.2]
gammas = [0.5e-6, 1e-6, 0.5e-5]
lambdas = [0.01]#np.logspace(-3, 1, 2)

ws = []
ratio_err_trains = []
ratio_err_tests = []

print("There will be {exp} experiments".format(exp=len(degrees)*len(split_ratios)*len(gammas)*len(lambdas)))


for idx0, split_ratio in enumerate(split_ratios):
    for idx1, degree in enumerate(degrees):
        for idx2, gamma in enumerate(gammas):
            for idx3, lambda_ in enumerate(lambdas):
                for r in range(len(x)):
                    w[r], ratio_error_train[r], ratio_error_test[r] = reg_logistic_regression_split(x[r], y[r], degree, split_ratio, gamma, lambda_, seed)
                ws.append(copy.copy(w))
                ratio_err_trains.append(0)
                ratio_err_tests.append(0)
                for r in range(len(ratio_error_train)):
                    ratio_err_trains[-1]+= ratio_error_train[r]*prop[r]
                    ratio_err_tests[-1] += ratio_error_test[r]*prop[r]
                print("{exp}: proportion={p}, degree={d}, gamma={g}, lambda={l}, Train err_ratio={er_tr:.3f}, Test err_ratio={er_te:.3f}".format(
                       exp=idx0*len(degrees)*len(gammas)*len(lambdas)
                          +idx1*len(gammas)*len(lambdas)
                          +idx2*len(lambdas)+idx3+1, 
                       p=split_ratio, d=degree, g=gamma, l=lambda_,
                       er_tr=ratio_err_trains[-1], er_te=ratio_err_tests[-1]))
            

There will be 12 experiments
1: proportion=0.2, degree=2, gamma=5e-07, lambda=0.01, Train err_ratio=0.207, Test err_ratio=0.209
2: proportion=0.2, degree=2, gamma=1e-06, lambda=0.01, Train err_ratio=0.202, Test err_ratio=0.199
3: proportion=0.2, degree=2, gamma=5e-06, lambda=0.01, Train err_ratio=0.189, Test err_ratio=0.190
4: proportion=0.2, degree=3, gamma=5e-07, lambda=0.01, Train err_ratio=0.204, Test err_ratio=0.206
5: proportion=0.2, degree=3, gamma=1e-06, lambda=0.01, Train err_ratio=0.197, Test err_ratio=0.198


KeyboardInterrupt: 

In [128]:
idx = np.argmin(ratio_err_tests)
w_reg_log_reg = ws[idx]


idx_split = int(idx/(len(degrees)*len(gammas)*len(lambdas)))
idx_degree_gamma_lamb = idx%(len(degrees)*len(gammas)*len(lambdas))
idx_degree = int(idx_degree_gamma_lamb/(len(gammas)*len(lambdas)))
idx_gamma_lamb = idx_degree_gamma_lamb%(len(gammas)*len(lambdas))
idx_gamma = int(idx_gamma_lamb/len(gammas))
idx_lamb = (idx_gamma_lamb%len(lambdas))


print("Take experiments {i} out of {tot} experiments, error tr={er_tr:.3f}, error test={er_te:.3f}, split_ratio={spl}, degree={deg}, gamma={g}".format(
           i=idx, tot=len(ws), er_tr=ratio_err_trains[idx], er_te=ratio_err_tests[idx], 
           spl=split_ratios[idx_split],  deg=degrees[idx_degree], g=gammas[idx_gamma]))


Take experiments 2 out of 5 experiments, error tr=0.189, error test=0.190, split_ratio=0.2, degree=2, gamma=5e-07


In [12]:
z = [1, 0, 2, 3]
print(np.argmin(z))

1


# Cross-validation (with k-fold)

In [64]:
if data_subset:
    r=8
else:
    r=1

loss=[0]*r
w=[0]*r
w_LS=[0]*r
w_LS_GD=[0]*r
w_LS_SGD=[0]*r
w_LS_degree=[0]*r
w_ridge_reg=[0]*r
w_LR=[0]*r
phi=[0]*r

ratio_error_train=[0]*r
ratio_error_test=[0]*r

k_indices=[0]*r

## Least squares

In [76]:
seed = 6
degrees = [1, 3, 7, 11, 12, 13, 14]
exp = 0

loss_tr = []
loss_te = []
ratio_err_trains = []
ratio_err_tests  = []
ws = []
deg=[]

k_fold = 4

for r in range(len(x)):
    k_indices[r] = build_k_indices(y[r], k_fold, seed)

for degree in degrees:
    for r in range(len(x)):
        phi[r] = build_poly(x[r], degree)
        

    ratio_err_trains_k_fold = [0]*k_fold
    ratio_err_tests_k_fold  = [0]*k_fold
    ws_k_fold = []
    for k in range(k_fold):
        for r in range(len(x)):
            # split the data, and return train and test data: TODO
            phi_train, phi_test, y_train, y_test = split_data_k_fold(phi[r], y[r], k_indices[r], k)
            
            # calcualte weight through least square.: TODO
            w[r],_ = least_squares(y_train, phi_train)
            
            _, ratio_error_train = compute_classification_error(y_train, phi_train, w[r], logistic_reg=False)
            _, ratio_error_test = compute_classification_error(y_test, phi_test, w[r], logistic_reg=False)
            
            ratio_err_trains_k_fold[k] += ratio_error_train*prop[r]
            ratio_err_tests_k_fold[k]  += ratio_error_test*prop[r]

        ws_k_fold.append(copy.copy(w))

 
    idx = np.argmin(ratio_err_tests_k_fold) # get best experiment
    ws.append(ws_k_fold[idx])
    ratio_err_trains.append(ratio_err_trains_k_fold[idx])
    ratio_err_tests.append(ratio_err_tests_k_fold[idx])
    
        
    deg.append(degree)
        
    print("{e}: degree={d}, Train err_ratio={er_tr:.3f}, Test err_ratio={er_te:.3f}".format(
        e=exp, d=degree,er_tr=ratio_err_trains[exp], er_te=ratio_err_tests[exp]))
    exp += 1


0: degree=1, Train err_ratio=0.235, Test err_ratio=0.234
1: degree=3, Train err_ratio=0.194, Test err_ratio=0.194
2: degree=7, Train err_ratio=0.175, Test err_ratio=0.178
3: degree=11, Train err_ratio=0.168, Test err_ratio=0.170
4: degree=12, Train err_ratio=0.168, Test err_ratio=0.172
5: degree=13, Train err_ratio=0.167, Test err_ratio=0.172
6: degree=14, Train err_ratio=0.237, Test err_ratio=0.239


In [77]:
idx = np.argmin(ratio_err_tests) # get best experiment
w_LS_degree_CV = ws[idx]


print("Take experiment {i} out of {tot} experiments, degree={d}, err_ratio={err}".format(
           i=idx, tot=len(ws), d=deg[idx], err=ratio_err_tests[idx]))

Take experiment 3 out of 7 experiments, degree=11, err_ratio=0.17028090316174446


## reg_logistic_regression

In [21]:
def reg_logistic_regression_split_k_fold(x, y, degree, gamma, lambda_, k_indices, k):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data_k_fold(x, y, k_indices, k)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)

    w_initial = np.array([0]*(phi_train.shape[1]))
    max_iters = 1000


    # calcualte weight through least square.: TODO
    w,loss_tr = reg_logistic_regression(y_train, phi_train, lambda_, w_initial, max_iters, gamma)

    loss_te = compute_logistic_loss(y_test, phi_test, w)
    # compute error and loss for train and test data
    error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
    error, ratio_error_test = compute_classification_error(y_test, phi_test, w)


    #print("proportion={p}, degree={d}, gamma={g}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
    #      p=ratio, d=degree, g=gamma, tr=loss_tr, te=loss_te, er_tr=ratio_error_train, er_te=ratio_error_test))
    return w, ratio_error_train, ratio_error_test


seed = 6
degrees = [1, 2, 3, 4]
gammas = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
lambdas = [0.001, 0.01, 0.1, 1, 10]


ws = []
ratio_err_trains = []
ratio_err_tests = []

k_indices=[0]*8
for r in range(len(x)):
    k_indices[r] = build_k_indices(y[r], k_fold, seed)

k_fold = 3
print(len(k_indices[0]))

print("There will be {exp} experiments".format(exp=len(degrees)*k_fold*len(gammas)*len(lambdas)))

for idx1, degree in enumerate(degrees):
    for idx2, gamma in enumerate(gammas):
        for idx3, lambda_ in enumerate(lambdas):
            for k in range(k_fold):
                for r in range(len(x)):
                    k_indices = build_k_indices(y[r], k_fold, seed)
                    #tdiprint(r, np.asarray(y[r]).shape, np.asarray(y[r]).shape)
                    w[r], ratio_error_train[r], ratio_error_test[r] = reg_logistic_regression_split_k_fold(
                        x[r], y[r], degree, gamma, lambda_, k_indices, k)
                ws.append(copy.copy(w))
                ratio_err_trains.append(0)
                ratio_err_tests.append(0)
                for r in range(len(ratio_error_train)):
                    ratio_err_trains[-1]+= ratio_error_train[r]*prop[r]
                    ratio_err_tests[-1] += ratio_error_test[r]*prop[r]
                print("{exp}: degree={d}, gamma={g}, lambda={l}, k={k}/{k_f}, Train err_ratio={er_tr:.3f}, Test err_ratio={er_te:.3f}".format(
                       exp=idx1*len(gammas)*len(lambdas)*k+idx2*len(lambdas)*k+idx3*k+k, 
                       d=degree, g=gamma, k=k+1, k_f=k_fold, l=lambda_,
                       er_tr=ratio_err_trains[-1], er_te=ratio_err_tests[-1]))
print("done")


1
There will be 240 experiments


KeyboardInterrupt: 

# Perform estimation on evaluation set (for kaggle)
Une fois qu'on a le bon vecteur de poids, il suffit de runner ce code pour avoir le fichier de sortie.

In [9]:
ids_to_write = np.array([])
y_to_write = np.array([])

for r in range(len(x_ukn)):
    tx_ukn[r] = build_poly(x_ukn[r], 3) # put the right degree
    w_selected = w_LR_CV[r]             # put the desired weight vector
    y_ukn[r] = predict_labels(w_selected, tx_ukn[r], logistic_reg=True) # pay attention to the value of the last argument
    y_ukn[r][y_ukn[r]==0] = -1
    
    ids_to_write = np.concatenate([ids_to_write, ids_ukn[r]], axis=0)
    y_to_write = np.concatenate([y_to_write, y_ukn[r]], axis=0)
    
create_csv_submission(ids_to_write, y_to_write, 'submission.csv')